In [7]:
import sqlite3
from werkzeug.security import generate_password_hash
import pandas as pd

In [14]:
conn=sqlite3.connect('testFlask\db.sqlite')
c = conn.cursor()

In [15]:
df = pd.read_csv('parser/users.csv', sep=';')[['id', 'email', 'password', 'surname', 'name']].astype('str')
df['password'] = list(map(lambda x: generate_password_hash(x, method='sha256'), df['password']))
c.executemany("INSERT INTO user VALUES(?,?,?,?,?)", df.values)

In [4]:
df = pd.read_csv('parser/rooms.csv', sep=';')[['id', 'name', 'type', 'x', 'y', 'floor']].astype('str')
c.executemany("INSERT INTO room VALUES(?,?,?,?,?,?)", df.values)

In [5]:
df = pd.read_csv('parser/places.csv', sep=';')[['id', 'number', 'user_id', 'room_id']].astype('str')
c.executemany("INSERT INTO place VALUES(?,?,?,?)", df.values)

In [16]:
c.close()
conn.commit()
conn.close()

In [ ]:
c.execute('select * from room')
c.fetchall()